In [2]:
import os
import numpy as np
from PIL import Image
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

dataset = []
labels = []

LEXICON = "0123456789abcdefghijklmnopqrstuvwxyz"
batch_size = 64
epochs = 20
img_rows, img_cols = 28, 28
num_classes = len(LEXICON)
input_shape = (img_rows, img_cols, 1)

for filename in os.listdir("secondary_dataset"):
    if filename.endswith(".jpg"):
        img_path = os.path.join("secondary_dataset", filename)
        im = Image.open(img_path).convert('L')
        dataset.append(np.asarray(im, dtype=np.float32))
        label = filename.replace(".jpg", "").split("-")[0]
        labels.append(LEXICON.index(label))

In [3]:
display(Image.fromarray(np.array(dataset[1], dtype=np.uint8)))
display(labels[1])

13

In [4]:
how_many_training_samples = 278

x_train = np.array(dataset[:how_many_training_samples])
y_train = labels[:how_many_training_samples]

x_test = np.array(dataset[how_many_training_samples:])
y_test = labels[how_many_training_samples:]

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (278, 28, 28, 1)
278 train samples
250 test samples


In [6]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [10]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Train on 278 samples, validate on 250 samples
Epoch 1/20
278/278 [==============================] - 1s 2ms/step - loss: 3.5954 - accuracy: 0.0540 - val_loss: 3.4107 - val_accuracy: 0.1080
Epoch 2/20
278/278 [==============================] - 0s 1ms/step - loss: 3.2727 - accuracy: 0.1727 - val_loss: 3.1075 - val_accuracy: 0.2200
Epoch 3/20
278/278 [==============================] - 0s 991us/step - loss: 2.8804 - accuracy: 0.3094 - val_loss: 2.5174 - val_accuracy: 0.3600
Epoch 4/20
278/278 [==============================] - 0s 1ms/step - loss: 2.1757 - accuracy: 0.4676 - val_loss: 1.8833 - val_accuracy: 0.5680
Epoch 5/20
278/278 [==============================] - 0s 1ms/step - loss: 1.5281 - accuracy: 0.6295 - val_loss: 1.3659 - val_accuracy: 0.6360
Epoch 6/20
278/278 [==============================] - 0s 947us/step - loss: 1.1178 - accuracy: 0.7446 - val_loss: 0.7172 - val_accuracy: 0.8800
Epoch 7/20
278/278 [==============================] - 0s 974us/step - loss: 0.5893 - accuracy: 0.8

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)

In [12]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.11668883182480931
Test accuracy: 0.9879999756813049


In [13]:
model.save("recognizer.h5")